In [5]:
%pip install --upgrade numpy -q
%pip install --upgrade scikit-image -q


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 2.3.3 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install easyocr -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 2.2.6 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [49]:
import easyocr

reader = easyocr.Reader(['en'], gpu=True) # Set gpu=False if no GPU is available

results = reader.readtext('/Users/saurabhsachingosavi/video-kyc/5.jpg')

results

texts = [res[1] for res in results]

print(texts)

['314a7]fTDJT', 'HRT', 'Trerr', 'INCOME TAX DEPARTMENT', 'GOVT OF INDIA', 'VINAY CHANDRA ALOFS', 'SESHACHAR', '02/09/1981', 'Permanent Account Number', 'AQDPA68TSA', 'iknz Ckuku', 'Signature', 'L', 'ek']


In [31]:
import easyocr

reader = easyocr.Reader(['en'], gpu=True) # Set gpu=False if no GPU is available

results = reader.readtext('/Users/saurabhsachingosavi/video-kyc/5.jpg')

results

texts = [res[1] for res in results]

print(len(texts))



14


In [44]:
text_clean = text.strip()

if "Name" in text_clean or "Father" in text_clean:
    print("Kuch toh hai")
else:
    print("Kuch bhi nahi hai")

len(results)

for i, text in enumerate(results):
    text_clean = text.strip()
    if i+1 < len(results):
        print("True")

Kuch bhi nahi hai
True
True
True
True
True
True
True
True
True
True
True
True
True


In [39]:
import re

results = texts

final_data = {
    "Name": None,
    "Father's Name": None,
    "Date Of Birth": None,
    "PAN Number": None
}

for i, text in enumerate(results):
    text_clean = text.strip()

    # PAN number (regex: 5 letters + 4 digits + 1 letter)
    if final_data["PAN Number"] is None and len(text_clean) == 10 and text_clean.isalnum():
        final_data["PAN Number"] = text_clean

    # Date of Birth (dd/mm/yyyy)
    if final_data["Date Of Birth"] is None and re.match(r'^\d{2}/\d{2}/\d{4}$', text_clean):
        final_data["Date Of Birth"] = text_clean

    # Name (look for keyword "Name")
    if "Name" in text_clean and "Father" not in text_clean and i+1 < len(results):
        final_data["Name"] = results[i+1].strip()

    # Father's Name
    if "Father" in text_clean and i+1 < len(results):
        final_data["Father's Name"] = results[i+1].strip()

print(final_data)


{'Name': None, "Father's Name": None, 'Date Of Birth': '03/12/1982', 'PAN Number': 'AFEPT9994D'}


In [48]:
import re

def extract_pan_details(results):
    final_data = {
        "Name": None,
        "Father's Name": None,
        "Date Of Birth": None,
        "PAN Number": None
    }

    # Join all text blocks into one searchable string
    text_blocks = [t.strip() for t in results if isinstance(t, str)]
    joined_text = " ".join(text_blocks)

    # PAN Number: 5 letters + 4 digits + 1 letter
    pan_match = re.search(r'\b([A-Z]{5}[0-9]{4}[A-Z])\b', joined_text)
    if pan_match:
        final_data["PAN Number"] = pan_match.group(1)

    # Date of Birth: dd/mm/yyyy
    dob_match = re.search(r'\b(\d{2}/\d{2}/\d{4})\b', joined_text)
    if dob_match:
        final_data["Date Of Birth"] = dob_match.group(1)

    # Search by keywords for Name
    for i, text in enumerate(text_blocks):
        if re.search(r'\bName\b', text, re.IGNORECASE) and "Father" not in text:
            if i+1 < len(text_blocks):
                final_data["Name"] = text_blocks[i+1]
        if re.search(r'Father', text, re.IGNORECASE):
            if i+1 < len(text_blocks):
                final_data["Father's Name"] = text_blocks[i+1]

    return final_data


# Example usage
results = texts
print(extract_pan_details(results))


{'Name': None, "Father's Name": None, 'Date Of Birth': '02/09/1981', 'PAN Number': None}


In [68]:
import easyocr

reader = easyocr.Reader(['en'], gpu=True) # Set gpu=False if no GPU is available

results = reader.readtext('/Users/saurabhsachingosavi/video-kyc/4_contrast.jpg')

results

texts = [res[1] for res in results]

print(texts)

['HRA rarr', 'EHEx', 'f[', 'GOVT. OF INDIA', 'McoyEtax DEPARTMENT', 'SATYANARAYAN Bisoi', 'KUMUDA CHANDRA BISOI', '26/01/4981', 'Peniaflent Account Number', 'AVEPBO29SM', 'Bcf', 'Dtfer', 'Igpature', 'Te']


In [69]:
import re
from difflib import get_close_matches

results = texts

final_data = {
    "Name": None,
    "Father's Name": None,
    "Date Of Birth": None,
    "PAN Number": None
}

# Common PAN card headers to ignore
ignore_words = [
    "INCOME TAX DEPARTMENT",
    "GOVT: OF INDIA",
    "GOVT OF INDIA",
    "PERMANENT ACCOUNT NUMBER CARD",
    "INCOME TAX",
    "GOVERNMENT OF INDIA"
]

name_candidates = []

for text in results:
    text_clean = text.strip()

    # PAN number (regex: 5 letters + 4 digits + 1 letter)
    if final_data["PAN Number"] is None and re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$', text_clean):
        final_data["PAN Number"] = text_clean

    # Date of Birth (dd/mm/yyyy)
    if final_data["Date Of Birth"] is None and re.match(r'^\d{2}/\d{2}/\d{4}$', text_clean):
        final_data["Date Of Birth"] = text_clean

    # Candidate for Name/Father’s Name (all caps, 2+ words, not in ignore list)
    is_upper_name = text_clean.isupper() and len(text_clean.split()) >= 2

    # Fuzzy match with ignore list
    ignore_match = get_close_matches(text_clean, ignore_words, n=1, cutoff=0.5)

    if is_upper_name and not ignore_match:
        name_candidates.append(text_clean)

# Assign names if candidates found
if name_candidates:
    final_data["Name"] = name_candidates[0]
    if len(name_candidates) > 1:
        final_data["Father's Name"] = name_candidates[1]

print(final_data)


{'Name': 'KUMUDA CHANDRA BISOI', "Father's Name": None, 'Date Of Birth': '26/01/4981', 'PAN Number': None}
